## Import Libraries

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd 
import PIL  
import tensorflow as tf
import itertools

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import os  
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from keras.callbacks import EarlyStopping, ReduceLROnPlateau  

from glob import glob
import pathlib

## Reading Data

In [ ]:
data_dir = '/kaggle/input/plantvillage-dataset'
color_dir = os.path.join(data_dir, 'color')
color_dir = pathlib.Path(color_dir)  

In [ ]:
#Sample leaves
c_Apple__healthy = list(color_dir.glob('Apple___healthy/*'))
print('Apple__healthy')
display(PIL.Image.open(str(c_Apple__healthy[0])))
display(PIL.Image.open(str(c_Apple__healthy[7])))

## Define functions to 
* make filepath-label-dataframe
* split data to train, valid and test
* create generators from the train, valid and test data. 
* show sample images using the generators. 

In [ ]:
# Generate data paths and labels dataframe. 
def make_path_label_df(data_dir):
    '''
    - takes the path to the data directory  
    - returns a dataframe: with filepath and labels for each image.'''
    
    filepaths = []
    labels = []

    folders = os.listdir(data_dir)
    for folder in folders:
        folderpath = os.path.join(data_dir, folder)
        files = os.listdir(folderpath)
        for file in files:
            filepath = os.path.join(folderpath, file)
            filepaths.append(filepath)
            labels.append(folder)

    Filepath_series = pd.Series(filepaths, name= 'filepaths')
    Label_series = pd.Series(labels, name='labels')
    dataframe = pd.concat([Filepath_series, Label_series], axis= 1)
    return dataframe

# Split dataframe to train, valid, and test
def split_to_train_valid_test(data_dir):
    '''
    - takes the path to the data directory
    - returns the three dataframes(train, valid and test respectively) containing the file paths and labels for each set.'''
    # train dataframe
    df = make_path_label_df(data_dir)
    print('Displaying sample of Dataframe with filenames and labels')
    display(df.sample(10))
    strat = df['labels']                                                
    train_df, dummy_df = train_test_split(df,  train_size= 0.8, shuffle= True, random_state= 42, stratify= strat)     #Note stratify=strat

    # valid and test dataframe
    strat = dummy_df['labels']
    valid_df, test_df = train_test_split(dummy_df,  train_size= 0.5, shuffle= True, random_state= 42, stratify= strat)
    print('Train, valid, test dataframe created:')
    print('Train dataframe sample: ')
    display(train_df.sample(2))
    print('|'+'-'*80+'|')
    print('valid dataframe sample: ')
    display(valid_df.sample(2))
    print('|'+'-'*80+'|')
    print('test dataframe sample: ')
    display(test_df.sample(2))
    print('|'+'-'*80+'|')

    return train_df, valid_df, test_df

In [ ]:
#create generators for given dataframes. 
def make_generators(train_df, valid_df, test_df, batch_size):
    '''
    This function takes train, validation, and test dataframe and fit them into image data generator, because model takes data from image data generator.
    Image data generator converts images into tensors. '''

    # define model parameters
    img_size = (224, 224)
    channels = 3      
    color = 'rgb'
    img_shape = (224, 224, 3)

    #test_batch_size and test_steps
    test_df_length = len(test_df)    
    test_batch_size = 8               
    test_steps = test_df_length // test_batch_size
    

    #Function to return image as it is
    def identity(img):
        return img

    t_gen = ImageDataGenerator(preprocessing_function= identity, horizontal_flip= True)              
    vt_gen = ImageDataGenerator(preprocessing_function= identity)                    

    train_gen = t_gen.flow_from_dataframe( train_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',     
                                        color_mode= color, shuffle= True, batch_size= batch_size)

    valid_gen = vt_gen.flow_from_dataframe( valid_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                        color_mode= color, shuffle= True, batch_size= batch_size)

    # Note: we will use custom test_batch_size, and make shuffle= false
    test_gen = vt_gen.flow_from_dataframe( test_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                        color_mode= color, shuffle= False, batch_size= test_batch_size)       

    return train_gen, valid_gen, test_gen

In [ ]:
def show_samples(generator):
    '''
    This function take the data generator and show sample of the images
    '''

    # return classes , images to be displayed
    g_dict = generator.class_indices        
    classes = list(g_dict.keys())     
    images, labels = next(generator)  

    # calculate number of displayed samples
    length = len(labels)       
    sample = min(length, 20)    

    plt.figure(figsize= (25, 20))

    for i in range(sample):
        #show image
        plt.subplot(4, 5, (i + 1))
        image = images[i] / 255       # scale pixels
        plt.imshow(image)
        # get class of image
        index = np.argmax(labels[i])  
        class_name = classes[index]   
        plt.title(class_name, color= 'purple', fontsize= 12)
        plt.subplots_adjust(hspace=0.1, wspace=0.6)       
        plt.axis('off')
    plt.show()

## Creating generators

In [ ]:
data_directory = color_dir 

# Get splitted data
train_df, valid_df, test_df = split_to_train_valid_test(data_directory)

# Get Generators
train_gen, valid_gen, test_gen = make_generators(train_df, valid_df, test_df, batch_size = 32)

## Visualizing image samples

In [ ]:
show_samples(train_gen)

## Define and compile model
* We will use 'EfficientNetB3' as the base model. 
* Lower layers will be freezed.
* Upper 80 layers will be trained.

In [ ]:
# Create Model Structure
img_shape=(224,224,3)
class_count = len(list(train_gen.class_indices.keys())) 

# For transfer learning, will use efficientnetb3 from EfficientNet family.
base_model = tf.keras.applications.efficientnet.EfficientNetB3(include_top= False, weights= "imagenet", input_shape= img_shape, pooling= 'max')            #transfer learning
base_model.trainable = True

# Freeze all layers except for the last 80 layers.
for layer in base_model.layers[:-80]:
    layer.trainable = False

model = Sequential([
    base_model,
    layers.BatchNormalization(axis= -1, momentum= 0.99, epsilon= 0.001),
    layers.Dense(256, kernel_regularizer= keras.regularizers.l2(l= 0.01), activity_regularizer= keras.regularizers.l1(0.001),
                bias_regularizer= keras.regularizers.l1(0.001), activation= 'relu'),
    layers.Dropout(rate= 0.4, seed= 42),
    layers.Dense(class_count, activation= 'softmax')
])

model.compile(keras.optimizers.Adamax(learning_rate= 0.0005), loss= 'categorical_crossentropy', metrics= ['accuracy'])       

model.summary()

## Define callbacks and fit the model

In [ ]:
batch_size = 32   
epochs = 10                 
lr_patience = 1   
early_stop_patience = 2   
factor = 0.5   

# Define the callbacks
early_stop = EarlyStopping(monitor='val_loss',
                           patience=early_stop_patience,
                           verbose=1, 
                           mode='min', 
                           baseline=None,
                           restore_best_weights=True
                          )
lr_reduction_on_plateau = ReduceLROnPlateau(monitor='accuracy',
                                            patience=lr_patience,
                                            factor=factor)
callback_list = [early_stop, lr_reduction_on_plateau]          

In [ ]:
history = model.fit(x= train_gen, epochs= epochs, verbose=1, callbacks= callback_list,
                    validation_data= valid_gen, validation_steps= None, shuffle= False)

## Define functions to 
* Plot training history
* Plot Confusion matrix 
* Plot Classification Report
* Visualize Predictions

In [ ]:
def plot_training_history(history_):               
    '''
    This function receives a trained model's history as input and generates a plot that displays the 
    accuracy and loss histories of the model, highlighting the best epoch for both metrics.
    '''

    # Define needed variables
    tr_acc = history_.history['accuracy']
    tr_loss = history_.history['loss']
    val_acc = history_.history['val_accuracy']
    val_loss = history_.history['val_loss']
    
    min_loss_index = np.argmin(val_loss)
    max_acc_index = np.argmax(val_acc)
    val_lowest = val_loss[min_loss_index]
    acc_highest = val_acc[max_acc_index]
    
    Epoch_numbers = [i+1 for i in range(len(tr_acc))]
    
    loss_label = f'best epoch= {str(min_loss_index + 1)}'
    acc_label = f'best epoch= {str(max_acc_index + 1)}'

    # Plot training history
    sns.set_style("whitegrid")
    plt.figure(figsize=(20, 8))

    plt.subplot(1, 2, 1)
    sns.lineplot(x=Epoch_numbers, y=tr_loss, color='r', label='Training loss')
    sns.lineplot(x=Epoch_numbers, y=val_loss, color='b', label='Validation loss')
    plt.scatter(min_loss_index + 1, val_lowest, s= 250, c= 'green', alpha=0.3, label= loss_label)
    plt.title('Training and Validation Losses')
    plt.xlabel('Number of Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    sns.lineplot(x=Epoch_numbers, y=tr_acc, color='r', label='Training Accuracy')
    sns.lineplot(x=Epoch_numbers, y=val_acc, color='b', label='Validation Accuracy')
    plt.scatter(max_acc_index + 1 , acc_highest, s= 250, c= 'green', alpha=0.3, label= acc_label)
    plt.title('Training and Validation Accuracies')
    plt.xlabel('Number of Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()


In [ ]:
def plot_confusion_matrix(generator, y_true, y_pred):
    #let's plot confusion matrix
    import itertools
    sns.set_style('white')
    # Get the class indices and labels
    g_dict = generator.class_indices
    classes = list(g_dict.keys())

    cm = confusion_matrix(y_true, y_pred)

    # Plot the confusion matrix
    plt.figure(figsize=(25,25))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Greens, vmin=0, vmax=1)   #--> TRY IT..
    plt.title('Confusion matrix')
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    # Normalize the confusion matrix
    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    text_color = (0.5, 0.7, 0.5)

    # Plot the normalized confusion matrix
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, "{:.2f}".format(cm_norm[i, j]),
                 horizontalalignment="center",
                 color=text_color)                

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()


In [ ]:
def plot_classification_report(y_true, y_pred, classes):
    report = classification_report(y_true, y_pred, target_names=classes, output_dict=True)
    report_df = report_df = pd.DataFrame(report)      
    # Plot heatmap
    plt.figure(figsize=(6, 15))
    ax = sns.heatmap(report_df.iloc[:-1, :-3].T, cmap='Blues', annot=True, fmt='.2f', cbar=False)
    ax.set_title('Classification Report')
    ax.set_xlabel('Metrics')
    ax.set_ylabel('Classes')
    plt.xticks(rotation=90, ha='right')
    plt.show()


In [ ]:
def visualize_predictions(test_generator, pred_generator):
    '''This function is suited to test_generator that has batch_size of 8.'''
    #make a list of 8 next predictions using generator
    pred_values = []
    for i in range(8):                                 
        pred_values.append(next(pred_generator))
    
    # return classes , images to be displayed
    g_dict = test_generator.class_indices        
    classes = list(g_dict.keys())
    images, labels = next(test_generator)

    #specify length
    length = 8

    plt.figure(figsize= (28, 12))

    for i in range(length):
        #show image
        plt.subplot(2, 4, (i + 1))
        image = images[i] / 255       # scale pixels
        plt.imshow(image)
        # get class of image
        # get class of image
        index_true = np.argmax(labels[i])  
        class_name_true = classes[index_true] 
        class_name_pred = classes[pred_values[i]]
        plt.title(f'Actual: {class_name_true}\nPredicted: {class_name_pred}', color= 'purple', fontsize= 14)
        plt.subplots_adjust(hspace=0.3, wspace=0.1)       
        plt.axis('off')
    plt.show()
    

## Plot the training history. 

In [ ]:
#Let's use the function defined above.
plot_training_history(history)

## Evaluate scores on train, valid and test sets.

In [ ]:
test_df_length = len(test_df)
train_df_length = len(train_df)
valid_df_length = len(valid_df)
test_batch_size = 8                 
test_steps = test_df_length // test_batch_size    
train_steps = train_df_length // 32
valid_steps = valid_df_length // 32

#evaluate scores
train_score = model.evaluate(train_gen, steps= train_steps, verbose= 1)
valid_score = model.evaluate(valid_gen, steps= valid_steps, verbose= 1)
test_score = model.evaluate(test_gen, steps= test_steps, verbose= 1)

# Print scores
print('|'+'-' * 40+'|')
print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('|'+'-' * 40+'|')

print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print('|'+'-' * 40+'|')

print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

## Plot Classification Report and Confusion Matrix 

In [ ]:
y_true = test_gen.classes
preds = model.predict(test_gen)       
y_pred = np.argmax(preds, axis=1)

g_dict = test_gen.class_indices
classes = list(g_dict.keys())

In [ ]:
plot_classification_report(y_true, y_pred, classes)

In [ ]:
plot_confusion_matrix(test_gen, y_true, y_pred)    

## Visualize Predictions

In [ ]:
test_gen.reset()
pred_gen = (x for x in y_pred)

In [ ]:
visualize_predictions(test_gen, pred_gen)

In [ ]:
visualize_predictions(test_gen, pred_gen)